# 使用するデータの紹介と読み込み

## データ読み込み

In [1]:
# https://portal.edirepository.org/nis/mapbrowse?packageid=knb-lter-pal.219.5 にあるコードを改変
import pandas as pd

uris = [
    # Adelie
    "https://pasta.lternet.edu/package/data/eml/knb-lter-pal/219/5/002f3893385f710df69eeebe893144ff",
    # Gentoo
    "https://pasta.lternet.edu/package/data/eml/knb-lter-pal/220/7/e03b43c924f226486f2f0ab6709d2381",
    # Chinstrap
    "https://pasta.lternet.edu/package/data/eml/knb-lter-pal/221/8/fe853aa8f7a59aa84cdd3197619ef462",
]


def read_data(uri):
    return pd.read_csv(
        uri,
        sep=",",
        quotechar='"',
        usecols=[  # ・・①
            "Species",
            "Island",
            "Individual ID",
            "Date Egg",
            "Culmen Length (mm)",
            "Culmen Depth (mm)",
            "Flipper Length (mm)",
            "Body Mass (g)",
            "Sex",
            "Comments",
        ],
        na_values=".",  # 追加で欠損値として扱う文字列を指定
    ).rename(  # ・・②
        columns={
            "Individual ID": "Individual_ID",
            "Date Egg": "Date_Egg",
            "Culmen Length (mm)": "Culmen_Length",
            "Culmen Depth (mm)": "Culmen_Depth",
            "Flipper Length (mm)": "Flipper_Length",
            "Body Mass (g)": "Body_Mass",
        }
    )


df = pd.concat([read_data(uri) for uri in uris], ignore_index=True)  # ・・③

## 読み込み結果の簡単なチェック

In [2]:
df.shape

(344, 10)

In [3]:
# 先頭を表示
df.head()

,Species,Island,Individual_ID,Date_Egg,Culmen_Length,Culmen_Depth,Flipper_Length,Body_Mass,Sex,Comments
0,Adelie Penguin (Pygoscelis adeliae),Torgersen,N1A1,2007-11-11,39.1,18.7,181.0,3750.0,MALE,Not enough blood for isotopes.
1,Adelie Penguin (Pygoscelis adeliae),Torgersen,N1A2,2007-11-11,39.5,17.4,186.0,3800.0,FEMALE,NaN
2,Adelie Penguin (Pygoscelis adeliae),Torgersen,N2A1,2007-11-16,40.3,18.0,195.0,3250.0,FEMALE,NaN
3,Adelie Penguin (Pygoscelis adeliae),Torgersen,N2A2,2007-11-16,NaN,NaN,NaN,NaN,NaN,Adult not sampled.
4,Adelie Penguin (Pygoscelis adeliae),Torgersen,N3A1,2007-11-16,36.7,19.3,193.0,3450.0,FEMALE,NaN


In [4]:
# 最後を表示
df.tail()

,Species,Island,Individual_ID,Date_Egg,Culmen_Length,Culmen_Depth,Flipper_Length,Body_Mass,Sex,Comments
339,Chinstrap penguin (Pygoscelis antarctica),Dream,N98A2,2009-11-19,55.8,19.8,207.0,4000.0,MALE,NaN
340,Chinstrap penguin (Pygoscelis antarctica),Dream,N99A1,2009-11-21,43.5,18.1,202.0,3400.0,FEMALE,Nest never observed with full clutch.
341,Chinstrap penguin (Pygoscelis antarctica),Dream,N99A2,2009-11-21,49.6,18.2,193.0,3775.0,MALE,Nest never observed with full clutch.
342,Chinstrap penguin (Pygoscelis antarctica),Dream,N100A1,2009-11-21,50.8,19.0,210.0,4100.0,MALE,NaN
343,Chinstrap penguin (Pygoscelis antarctica),Dream,N100A2,2009-11-21,50.2,18.7,198.0,3775.0,FEMALE,NaN


In [5]:
df.dtypes

Species            object
Island             object
Individual_ID      object
Date_Egg           object
Culmen_Length     float64
Culmen_Depth      float64
Flipper_Length    float64
Body_Mass         float64
Sex                object
Comments           object
dtype: object

In [6]:
df["Date_Egg"] = pd.to_datetime(df.loc[:, "Date_Egg"], format="%Y-%m-%d")

In [7]:
df["Species"] = df.loc[:, "Species"].astype("category")
df["Island"] = df.loc[:, "Island"].astype("category")
df["Sex"] = df.loc[:, "Sex"].astype("category")

In [8]:
import re

RE_SHORT = re.compile(r"^\w+")  # 先頭から連続するUnicodeの単語文字にマッチ
df["Species_short"] = df.loc[:, "Species"].apply(
    lambda x: re.search(RE_SHORT, x)[0]
)

In [9]:
df.dtypes

Species                 category
Island                  category
Individual_ID             object
Date_Egg          datetime64[ns]
Culmen_Length            float64
Culmen_Depth             float64
Flipper_Length           float64
Body_Mass                float64
Sex                     category
Comments                  object
Species_short           category
dtype: object

In [10]:
df.loc[:, "Species_short"].cat.categories

Index(['Adelie', 'Chinstrap', 'Gentoo'], dtype='object')

## 読み込んだデータの保存

In [11]:
df.to_parquet("data/penguins.parquet")

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.